In [1]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import numpy as np
import more_itertools

In [2]:
A = ['B-Condition', 'I-Condition', 'B-Drug', 'O','O','B-Drug','I-Drug','I-Drug','O', 'O']
P = ['B-Condition', 'I-Condition', 'B-Temporal', 'O','O','O','I-Drug','O', 'O', 'O']

len(A), len(P)

(10, 10)

In [3]:
tag_idx = {'Drug': 0, 'O': 1, 'Device': 2, 'Negation': 3, 'Condition': 4, 'Qualifier': 5, 'Procedure': 6, 'Measurement': 7, 'Value': 8, 'Temporal': 9, 'PAD': 10, 'Observation': 11, 'Person': 12}

In [4]:
tag_values2idx = {i: t for i, t in enumerate(tag_idx)}
print(tag_values2idx)

{0: 'Drug', 1: 'O', 2: 'Device', 3: 'Negation', 4: 'Condition', 5: 'Qualifier', 6: 'Procedure', 7: 'Measurement', 8: 'Value', 9: 'Temporal', 10: 'PAD', 11: 'Observation', 12: 'Person'}


In [5]:
y_pred = [0, 2, 1, 3, 2]
y_true = [0, 1, 2, 3, 3]
accuracy_score(y_true, y_pred)

0.4

In [6]:
f1_score(y_true, y_pred, average='macro')
f1_score(y_true, y_pred, average='micro')

0.4000000000000001

In [7]:
tag2Dict = {'Qualifier': 0, 'Procedure': 1, 'Measurement': 2, 'Observation': 3, 'Condition': 4, 'Temporal': 5, 'Drug': 6, 'Person': 7, 'Value': 8, 'Device': 9, 'Negation': 10, 'O':11}
tag2Dict

{'Qualifier': 0,
 'Procedure': 1,
 'Measurement': 2,
 'Observation': 3,
 'Condition': 4,
 'Temporal': 5,
 'Drug': 6,
 'Person': 7,
 'Value': 8,
 'Device': 9,
 'Negation': 10,
 'O': 11}

In [8]:
def getPredictions(A, P):   
    def getEntityIndices(A):   
        entity_index = []
        for i, a in enumerate(A):
            if a.startswith('B-'):
                if i == len(A)-1:
                    entity_index.append((i,i,a[2:]))
                else:
                    for j, b in enumerate(A, start=i+1):
                        if A[j].startswith('I-'):
                            if j == (len(A)-1):
                                entity_index.append((i,j,a[2:]))
                                break
                            pass
                        else:
                            entity_index.append((i,j-1,a[2:]))
                            break
            elif a == 'O':
                if i == len(A)-1:
                    entity_index.append((i,i,a))
                else:
                    for j, b in enumerate(A, start=i+1):
                        if A[j] == 'O':
                            if j == (len(A)-1):
                                entity_index.append((i,j,a))
                                break
                            pass
                        else:
                            entity_index.append((i,j-1,a))
                            break

        return entity_index

    def removeMultipleOs(C):
        I = [i for i, c in enumerate(C) if c[2] == 'O']        
        grplist = [list(group) for group in more_itertools.consecutive_groups(I)]
        indexList = [grp[0] for grp in grplist]
        removeList = []
        for i, c in enumerate(C):
            if (c[2] == 'O') and (i not in indexList):
                removeList.append(i)
        C = [i for j, i in enumerate(C) if j not in removeList]
        return C

    def getTrueLabels(entity_index):
        true_label_entities = []
        for entities in entity_index:
            true_label_entities.append(entities[2])
        return true_label_entities


    def getPredLabels(P, entity_index):
        prediction_entities = []
        for j, entities in enumerate(entity_index):
            boolFindOverlap = correctOverlap = False
            temp = 'Blah'
            for i in range(entities[0],entities[1]+1):
                if P[i].startswith('B-') or P[i].startswith('I-'):
                    boolFindOverlap = True 
                    temp = P[i][2:]
                    if P[i][2:] == entities[2]:
                        correctOverlap = True
                        prediction_entities.append(entities[2])
                        break
                else:
                    continue
            if boolFindOverlap == False and correctOverlap == False:
                prediction_entities.append('O')
            elif boolFindOverlap == True and correctOverlap == False:
                prediction_entities.append(temp)

            assert (j+1)==len(prediction_entities)

        return prediction_entities
    
    entity_index = getEntityIndices(A)
    entity_index = removeMultipleOs(entity_index)
    true_label_entities = getTrueLabels(entity_index)
    prediction_entities = getPredLabels(P, entity_index)
    
    assert len(true_label_entities) == len(prediction_entities)
    return true_label_entities, prediction_entities
    
true_label_entities, prediction_entities = getPredictions(A, P)
true_label_entities, prediction_entities

(['Condition', 'Drug', 'O', 'Drug', 'O'],
 ['Condition', 'Temporal', 'O', 'Drug', 'O'])

In [9]:
def getReports(true_label_entities, prediction_entities):

    def classifcationReport(true_label_entities, prediction_entities):
        CR = classification_report(true_label_entities, prediction_entities)
        CR_Dict = classification_report(true_label_entities, prediction_entities, output_dict=True)
        return CR, CR_Dict
    
    def getConfusionMatrix(true_label_entities, prediction_entities, CR_Dict):
        labels_ = list(CR_Dict.keys())[:-3]
        confusionMatrix = confusion_matrix(true_label_entities, prediction_entities, labels=labels_)
        return confusionMatrix
    
    def getAccuracy(confusionMatrix, CR_Dict):
        labels = list(CR_Dict.keys())[:-3]
        acc = np.diag(confusionMatrix)/confusionMatrix.sum(1)
        
        return labels, acc
    
    CR, CR_Dict = classifcationReport(true_label_entities, prediction_entities)
    confusionMatrix = getConfusionMatrix(true_label_entities, prediction_entities, CR_Dict)
    labels, acc = getAccuracy(confusionMatrix, CR_Dict)
    print(CR)

In [10]:
true_label_entities ,prediction_entities = getPredictions(A, P)
true_label_entities, prediction_entities

(['Condition', 'Drug', 'O', 'Drug', 'O'],
 ['Condition', 'Temporal', 'O', 'Drug', 'O'])

In [12]:
getReports(true_label_entities, prediction_entities)

              precision    recall  f1-score   support

   Condition       1.00      1.00      1.00         1
        Drug       1.00      0.50      0.67         2
           O       1.00      1.00      1.00         2
    Temporal       0.00      0.00      0.00         0

    accuracy                           0.80         5
   macro avg       0.75      0.62      0.67         5
weighted avg       1.00      0.80      0.87         5



/home/umar.salman/G42/nlp2sqlEnv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umar.salman/G42/nlp2sqlEnv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umar.salman/G42/nlp2sqlEnv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umar.salman/G42/nlp

In [14]:
CR = classification_report(true_label_entities, prediction_entities)
CR_Dict = classification_report(true_label_entities, prediction_entities, output_dict=True)
CR_Dict

/home/umar.salman/G42/nlp2sqlEnv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umar.salman/G42/nlp2sqlEnv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umar.salman/G42/nlp2sqlEnv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/umar.salman/G42/nlp

{'Condition': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1},
 'Drug': {'precision': 1.0,
  'recall': 0.5,
  'f1-score': 0.6666666666666666,
  'support': 2},
 'O': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 2},
 'Temporal': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0},
 'accuracy': 0.8,
 'macro avg': {'precision': 0.75,
  'recall': 0.625,
  'f1-score': 0.6666666666666666,
  'support': 5},
 'weighted avg': {'precision': 1.0,
  'recall': 0.8,
  'f1-score': 0.8666666666666666,
  'support': 5}}

In [15]:
list(CR_Dict.keys())[:-3]

['Condition', 'Drug', 'O', 'Temporal']

In [18]:
y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
confusionMatrix = confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"])
confusionMatrix

array([[2, 0, 0],
       [0, 0, 1],
       [1, 0, 2]])

In [19]:
print(np.diag(confusionMatrix)/confusionMatrix.sum(1))

[1.         0.         0.66666667]


In [ ]:
# https://github.com/umar1997/Courses/blob/main/Data%20Science/Linear_Regression_%26_Bag_Of_Words_Model.ipynb
# For SNS Plot

In [12]:
pred = t_label = [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
tag_values = ['O', 'Procedure', 'Value', 'Qualifier', 'Device', 'Observation', 'Measurement', 'PAD', 'Condition', 'Temporal', 'Negation', 'Drug', 'Person']


In [13]:
zipList = [(tag_values[p],tag_values[t])  for p, t in zip(pred, t_label) if tag_values[t]!= 'PAD']
pred, labels = zip(*zipList)
pred, labels = list(pred), list(labels)

ValueError: not enough values to unpack (expected 2, got 0)

In [14]:
zipList

[]